In [1]:
import torch
import numpy as np 
 
# =====================
# generate data
def gen_fake_data(hidden_weight, hidden_bias):
    def rand_5_numbers_from1to10():
        return np.random.permutation(10)[:5]

    x_train =  torch.Tensor([rand_5_numbers_from1to10() for i in range(60)]) 
    x_test = torch.Tensor([rand_5_numbers_from1to10() for i in range(60)]) 
    
    hidden_weight = torch.Tensor(hidden_weight).unsqueeze(-1) 
    hidden_bias = torch.Tensor([hidden_bias]) 

    y_train = torch.mm(x_train, hidden_weight) + hidden_bias
    y_test = torch.mm(x_test, hidden_weight) + hidden_bias
    return x_train, y_train, x_test, y_test
     
# gen training data sample 
x_train, y_train, x_test, y_test = gen_fake_data(hidden_weight=[1,3,2,4,5], hidden_bias=1)
print(x_train[:3])
print(y_train[:3])
# =====================

tensor([[2., 0., 1., 5., 4.],
        [0., 5., 3., 7., 6.],
        [5., 3., 4., 6., 8.]])
tensor([[45.],
        [80.],
        [87.]])


/tmp/ipykernel_2847809/2941274300.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  x_train =  torch.Tensor([rand_5_numbers_from1to10() for i in range(60)])


In [2]:
from torch.utils.data import DataLoader

# dataloader
# dataset_example should support operator index_selection for create the data_loader object
train_loader = DataLoader([(x_train[i], y_train[i]) for i in range(x_train.shape[0])], batch_size=10, collate_fn=None, shuffle=True)
test_loader = DataLoader([(x_test[i], y_test[i]) for i in range(x_test.shape[0])], batch_size=50, collate_fn=None, shuffle=True)

## Practice part 
- train a model to find hidden "weights and bias" from training data.
- **TODO**, practice with learning rate equal to 0.005, 0.02 0.01 to see the result how the model work 

In [3]:
# train a model to find hidden "weights and bias" from training data.
# TODO, practice with learning rate equal to 0.005, 0.02 0.01 to see the result how the model work 

# ============= ############## ===============
# USE LIBRARY
model = torch.nn.Linear(5,1)
model.cuda()

# loss also is supported by a library 
loss_computation = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.005)
# ============= ############## ===============

MAX_EPOCHS=5000
for epoch in range(MAX_EPOCHS):
    loss = 100000
    for batch in train_loader:

        x, y_gold = batch
  
        x = x.cuda()
        y_gold = y_gold.cuda()
        
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)


        # ============= ############## ===============
        # Compute and loss = average ((out_put - pred) ^ 2)
        loss = loss_computation(y_pred, y_gold)
        # ============= ############## ===============

        # perform a backward pass (backpropagation) => to compute the gradient values in Tensor weights
        loss.backward()

        # ============= ############## ===============
        # USE LIBRARY: 'model.parameters()' in stead of 'model.get_parameter()' is implemented by library, also return list of parameters: "weight" and "bias" 
        # Optimizer step(), this update gradient values to weights.
        optimizer.step() # instead of `param.add_(-lr * param.grad)` => update weight values
        optimizer.zero_grad() # instead of `param.grad.fill_(0)` => remove all the old gradient values in all Tensor weight
        # ============= ############## ===============
    
    if loss.item() < 0.001:
        break
    print('epoch/batch: ', epoch, ' loss: ', loss.item())

print(model.weight,  model.bias) 

epoch/batch:  0  loss:  45.8241081237793
epoch/batch:  1  loss:  5.250632286071777
epoch/batch:  2  loss:  2.9515390396118164
epoch/batch:  3  loss:  0.9043130874633789
epoch/batch:  4  loss:  0.17148271203041077
epoch/batch:  5  loss:  0.06080832704901695
epoch/batch:  6  loss:  0.02346581034362316
epoch/batch:  7  loss:  0.021725833415985107
epoch/batch:  8  loss:  0.013775328174233437
epoch/batch:  9  loss:  0.017068536952137947
epoch/batch:  10  loss:  0.013331432826817036
epoch/batch:  11  loss:  0.006817650981247425
epoch/batch:  12  loss:  0.00743310758844018
epoch/batch:  13  loss:  0.011507446877658367
epoch/batch:  14  loss:  0.005524070467799902
epoch/batch:  15  loss:  0.0071863518096506596
epoch/batch:  16  loss:  0.004175671376287937
epoch/batch:  17  loss:  0.006212280131876469
epoch/batch:  18  loss:  0.004982548300176859
epoch/batch:  19  loss:  0.00919510331004858
epoch/batch:  20  loss:  0.004378370475023985
epoch/batch:  21  loss:  0.007110916078090668
epoch/batch: 